In [ ]:
import pygame
from pygame.locals import *

pygame.init()
WIDTH, HEIGHT = 540, 600
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Sudoku")

FONT = pygame.font.SysFont("comicsans", 40)
SMALL_FONT = pygame.font.SysFont("comicsans", 20)

# Sample board (0 = empty)
initial_board = [
    [7, 8, 0, 4, 0, 0, 1, 2, 0],
    [6, 0, 0, 0, 7, 5, 0, 0, 9],
    [0, 0, 0, 6, 0, 1, 0, 7, 8],
    [0, 0, 7, 0, 4, 0, 2, 6, 0],
    [0, 0, 1, 0, 5, 0, 9, 3, 0],
    [9, 0, 4, 0, 6, 0, 0, 0, 5],
    [0, 7, 0, 3, 0, 0, 0, 1, 2],
    [1, 2, 0, 0, 0, 7, 4, 0, 0],
    [0, 4, 9, 2, 0, 6, 0, 0, 7]
]

class Grid:
    def __init__(self, rows, cols, width, height, board):
        self.rows = rows
        self.cols = cols
        self.width = width
        self.height = height
        self.board = board
        self.cubes = [[Cube(board[i][j], i, j, width, height) for j in range(cols)] for i in range(rows)]
        self.model = None
        self.selected = None

    def draw(self, win):
        gap = self.width / 9
        for i in range(self.rows + 1):
            thick = 4 if i % 3 == 0 else 1
            pygame.draw.line(win, (0, 0, 0), (0, i * gap), (self.width, i * gap), thick)
            pygame.draw.line(win, (0, 0, 0), (i * gap, 0), (i * gap, self.height), thick)

        for i in range(self.rows):
            for j in range(self.cols):
                self.cubes[i][j].draw(win)

    def click(self, pos):
        if pos[0] < self.width and pos[1] < self.height:
            gap = self.width / 9
            x = int(pos[0] // gap)
            y = int(pos[1] // gap)
            return (y, x)
        return None

    def select(self, row, col):
        for i in range(9):
            for j in range(9):
                self.cubes[i][j].selected = False
        self.cubes[row][col].selected = True
        self.selected = (row, col)

    def sketch(self, val):
        row, col = self.selected
        self.cubes[row][col].temp = val

    def place_number(self, val):
        row, col = self.selected
        if self.valid_move(val, (row, col)):
            self.cubes[row][col].value = val
            self.cubes[row][col].temp = 0
            self.board[row][col] = val
            return True
        return False

    def valid_move(self, num, pos):
        for i in range(9):
            if self.board[pos[0]][i] == num and pos[1] != i:
                return False
        for i in range(9):
            if self.board[i][pos[1]] == num and pos[0] != i:
                return False
        box_x = pos[1] // 3
        box_y = pos[0] // 3
        for i in range(box_y * 3, box_y * 3 + 3):
            for j in range(box_x * 3, box_x * 3 + 3):
                if self.board[i][j] == num and (i, j) != pos:
                    return False
        return True

    def is_full(self):
        return all(self.board[i][j] != 0 for i in range(9) for j in range(9))

    def solve_gui(self):
        find = self.find_empty()
        if not find:
            return True
        row, col = find
        for i in range(1, 10):
            if self.valid_move(i, (row, col)):
                self.board[row][col] = i
                self.cubes[row][col].value = i
                self.cubes[row][col].draw_change(WIN, True)
                pygame.display.update()
                pygame.time.delay(30)

                if self.solve_gui():
                    return True

                self.board[row][col] = 0
                self.cubes[row][col].value = 0
                self.cubes[row][col].draw_change(WIN, False)
                pygame.display.update()
                pygame.time.delay(30)
        return False

    def find_empty(self):
        for i in range(9):
            for j in range(9):
                if self.board[i][j] == 0:
                    return (i, j)
        return None

class Cube:
    def __init__(self, value, row, col, width, height):
        self.value = value
        self.temp = 0
        self.row = row
        self.col = col
        self.width = width
        self.height = height
        self.selected = False

    def draw(self, win):
        gap = self.width / 9
        x = self.col * gap
        y = self.row * gap

        if self.temp != 0 and self.value == 0:
            text = SMALL_FONT.render(str(self.temp), 1, (128, 128, 128))
            win.blit(text, (x + 5, y + 5))
        elif self.value != 0:
            text = FONT.render(str(self.value), 1, (0, 0, 0))
            win.blit(text, (x + gap/2 - text.get_width()/2, y + gap/2 - text.get_height()/2))

        if self.selected:
            pygame.draw.rect(win, (255, 0, 0), (x, y, gap, gap), 3)

    def draw_change(self, win, correct=True):
        gap = self.width / 9
        x = self.col * gap
        y = self.row * gap

        pygame.draw.rect(win, (255, 255, 255), (x, y, gap, gap), 0)
        text = FONT.render(str(self.value), 1, (0, 0, 0))
        win.blit(text, (x + gap/2 - text.get_width()/2, y + gap/2 - text.get_height()/2))
        color = (0, 255, 0) if correct else (255, 0, 0)
        pygame.draw.rect(win, color, (x, y, gap, gap), 3)

def redraw_window(win, board):
    win.fill((255, 255, 255))
    board.draw(win)

def main():
    board = Grid(9, 9, WIDTH, WIDTH, initial_board)
    key = None
    run = True
    while run:
        redraw_window(WIN, board)
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == QUIT:
                run = False

            if event.type == MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                clicked = board.click(pos)
                if clicked:
                    board.select(clicked[0], clicked[1])
                    key = None

            if event.type == KEYDOWN:
                if event.key == K_1: key = 1
                elif event.key == K_2: key = 2
                elif event.key == K_3: key = 3
                elif event.key == K_4: key = 4
                elif event.key == K_5: key = 5
                elif event.key == K_6: key = 6
                elif event.key == K_7: key = 7
                elif event.key == K_8: key = 8
                elif event.key == K_9: key = 9
                elif event.key == K_BACKSPACE:
                    if board.selected:
                        row, col = board.selected
                        board.cubes[row][col].value = 0
                        board.cubes[row][col].temp = 0
                        board.board[row][col] = 0
                elif event.key == K_RETURN:
                    if board.selected and key:
                        if board.place_number(key):
                            print(f"Placed {key}")
                        else:
                            print(f"{key} is not valid.")
                        key = None
                elif event.key == K_SPACE:
                    board.solve_gui()

        if board.selected and key is not None:
            board.sketch(key)

    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Placed 3
Placed 5
1 is not valid.
Placed 2
Placed 4
9 is not valid.
1 is not valid.
